In [1]:
# installing pyspark in colab (so i can use spark)
!pip -q install pyspark

# starting a spark session (this is like the entry point for pyspark)
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("vehicle_co2_task1") \
    .getOrCreate()

spark

In [2]:
# uploading the dataset from my computer to colab
from google.colab import files

uploaded = files.upload()

uploaded  # just to see the file name i uploaded


Saving co2.csv to co2.csv


{'co2.csv': b'Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)\r\nACURA,ILX,COMPACT,2,4,AS5,Z,9.9,6.7,8.5,33,196\r\nACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221\r\nACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6,5.8,5.9,48,136\r\nACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255\r\nACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244\r\nACURA,RLX,MID-SIZE,3.5,6,AS6,Z,11.9,7.7,10,28,230\r\nACURA,TL,MID-SIZE,3.5,6,AS6,Z,11.8,8.1,10.1,28,232\r\nACURA,TL AWD,MID-SIZE,3.7,6,AS6,Z,12.8,9,11.1,25,255\r\nACURA,TL AWD,MID-SIZE,3.7,6,M6,Z,13.4,9.5,11.6,24,267\r\nACURA,TSX,COMPACT,2.4,4,AS5,Z,10.6,7.5,9.2,31,212\r\nACURA,TSX,COMPACT,2.4,4,M6,Z,11.2,8.1,9.8,29,225\r\nACURA,TSX,COMPACT,3.5,6,AS5,Z,12.1,8.3,10.4,27,239\r\nALFA ROMEO,4C,TWO-SEATER,1.8,4,AM6,Z,9.7,6.9,8.4,34,193\r\nASTON MARTIN,DB9,MINICOMPACT,5.9,12,A6,Z,18

In [4]:
# reading the csv with spark
# header=True means first row has column names
# sep="," because it is a normal csv with commas
file_name = "co2.csv"

df_raw = spark.read.csv(
    file_name,
    header=True,
    inferSchema=True,
    sep=","
)

print("rows in raw df:", df_raw.count())
print("columns:", len(df_raw.columns))
df_raw.printSchema()
df_raw.show(5, truncate=False)


rows in raw df: 7385
columns: 12
root
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Vehicle Class: string (nullable = true)
 |-- Engine Size(L): double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Fuel Type: string (nullable = true)
 |-- Fuel Consumption City (L/100 km): double (nullable = true)
 |-- Fuel Consumption Hwy (L/100 km): double (nullable = true)
 |-- Fuel Consumption Comb (L/100 km): double (nullable = true)
 |-- Fuel Consumption Comb (mpg): integer (nullable = true)
 |-- CO2 Emissions(g/km): integer (nullable = true)

+-----+----------+-------------+--------------+---------+------------+---------+--------------------------------+-------------------------------+--------------------------------+---------------------------+-------------------+
|Make |Model     |Vehicle Class|Engine Size(L)|Cylinders|Transmission|Fuel Type|Fuel Consumption City (L/100 km)|Fuel Consumption Hwy (L/100

In [7]:
# renaming columns to simpler names (some have units and special characters)
# this makes the code easier for me later

new_cols = {
    "Make": "make",
    "Model": "model",
    "Vehicle Class": "vehicle_class",
    "Engine Size(L)": "engine_size_l",
    "Cylinders": "cylinders",
    "Transmission": "transmission",
    "Fuel Type": "fuel_type",
    "Fuel Consumption City (L/100 km)": "fuel_city_l100",
    "Fuel Consumption Hwy (L/100 km)": "fuel_hwy_l100",
    "Fuel Consumption Comb (L/100 km)": "fuel_comb_l100",
    "Fuel Consumption Comb (mpg)": "fuel_comb_mpg",
    "CO2 Emissions(g/km)": "co2_g_km"
}

df = df_raw
for old, new in new_cols.items():
    if old in df.columns:
        df = df.withColumnRenamed(old, new)

df.printSchema()
df.show(5, truncate=False)



root
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- vehicle_class: string (nullable = true)
 |-- engine_size_l: double (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- transmission: string (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- fuel_city_l100: double (nullable = true)
 |-- fuel_hwy_l100: double (nullable = true)
 |-- fuel_comb_l100: double (nullable = true)
 |-- fuel_comb_mpg: integer (nullable = true)
 |-- co2_g_km: integer (nullable = true)

+-----+----------+-------------+-------------+---------+------------+---------+--------------+-------------+--------------+-------------+--------+
|make |model     |vehicle_class|engine_size_l|cylinders|transmission|fuel_type|fuel_city_l100|fuel_hwy_l100|fuel_comb_l100|fuel_comb_mpg|co2_g_km|
+-----+----------+-------------+-------------+---------+------------+---------+--------------+-------------+--------------+-------------+--------+
|ACURA|ILX       |COMPACT      |2.0    

In [8]:
# checking MISSING VALUES per column
# i do a simple count of nulls because missing data can affect analysis and recommendations

from pyspark.sql.functions import col, sum as spark_sum, when

null_counts = df.select([
    spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df.columns
])

null_counts.show(truncate=False)


+----+-----+-------------+-------------+---------+------------+---------+--------------+-------------+--------------+-------------+--------+
|make|model|vehicle_class|engine_size_l|cylinders|transmission|fuel_type|fuel_city_l100|fuel_hwy_l100|fuel_comb_l100|fuel_comb_mpg|co2_g_km|
+----+-----+-------------+-------------+---------+------------+---------+--------------+-------------+--------------+-------------+--------+
|0   |0    |0            |0            |0        |0           |0        |0             |0            |0             |0            |0       |
+----+-----+-------------+-------------+---------+------------+---------+--------------+-------------+--------------+-------------+--------+



In [9]:
# cleaning the dataset
# i keep rows that have the main fields i need: make, model, vehicle class, fuel type and co2
# because missing these would make the recommender unreliable

from pyspark.sql.functions import trim

df_clean = df \
    .withColumn("make", trim(col("make"))) \
    .withColumn("model", trim(col("model"))) \
    .withColumn("vehicle_class", trim(col("vehicle_class"))) \
    .withColumn("fuel_type", trim(col("fuel_type"))) \
    .filter(col("make").isNotNull()) \
    .filter(col("model").isNotNull()) \
    .filter(col("vehicle_class").isNotNull()) \
    .filter(col("fuel_type").isNotNull()) \
    .filter(col("co2_g_km").isNotNull())

print("rows before:", df.count())
print("rows after cleaning:", df_clean.count())
df_clean.show(5, truncate=False)


rows before: 7385
rows after cleaning: 7385
+-----+----------+-------------+-------------+---------+------------+---------+--------------+-------------+--------------+-------------+--------+
|make |model     |vehicle_class|engine_size_l|cylinders|transmission|fuel_type|fuel_city_l100|fuel_hwy_l100|fuel_comb_l100|fuel_comb_mpg|co2_g_km|
+-----+----------+-------------+-------------+---------+------------+---------+--------------+-------------+--------------+-------------+--------+
|ACURA|ILX       |COMPACT      |2.0          |4        |AS5         |Z        |9.9           |6.7          |8.5           |33           |196     |
|ACURA|ILX       |COMPACT      |2.4          |4        |M6          |Z        |11.2          |7.7          |9.6           |29           |221     |
|ACURA|ILX HYBRID|COMPACT      |1.5          |4        |AV7         |Z        |6.0           |5.8          |5.9           |48           |136     |
|ACURA|MDX 4WD   |SUV - SMALL  |3.5          |6        |AS6         |Z    

In [10]:
# removing DUPLICATES
# some cars may appear more than once, so i drop duplicates based on key columns

key_cols = [c for c in ["make", "model", "vehicle_class", "engine_size_l", "transmission", "fuel_type"] if c in df_clean.columns]
df_clean = df_clean.dropDuplicates(key_cols)

print("rows after dropDuplicates:", df_clean.count())


rows after dropDuplicates: 3700


EXPLORATORY ANALYSIS ~~~~~~~~~~~~~~

In [11]:
# SUMMARY OF STATISTICS
# this helps me understand the distribution and if there are extreme values

df_clean.select("co2_g_km").describe().show()


+-------+------------------+
|summary|          co2_g_km|
+-------+------------------+
|  count|              3700|
|   mean|251.13837837837838|
| stddev| 59.88098304452377|
|    min|                96|
|    max|               522|
+-------+------------------+



In [12]:
# cars LOWEST co2 emissions
df_clean.select("make", "model", "vehicle_class", "fuel_type", "engine_size_l", "co2_g_km") \
    .orderBy(col("co2_g_km").asc()) \
    .show(10, truncate=False)

# cars HIGHEST co2 emissions
df_clean.select("make", "model", "vehicle_class", "fuel_type", "engine_size_l", "co2_g_km") \
    .orderBy(col("co2_g_km").desc()) \
    .show(10, truncate=False)


+-------+--------------+---------------------+---------+-------------+--------+
|make   |model         |vehicle_class        |fuel_type|engine_size_l|co2_g_km|
+-------+--------------+---------------------+---------+-------------+--------+
|HYUNDAI|IONIQ BLUE    |FULL-SIZE            |X        |1.6          |96      |
|HYUNDAI|IONIQ Blue    |FULL-SIZE            |X        |1.6          |96      |
|HYUNDAI|IONIQ         |FULL-SIZE            |X        |1.6          |103     |
|TOYOTA |Prius         |MID-SIZE             |X        |1.8          |105     |
|TOYOTA |Corolla Hybrid|COMPACT              |X        |1.8          |106     |
|TOYOTA |PRIUS c       |COMPACT              |X        |1.5          |108     |
|TOYOTA |Prius AWD     |MID-SIZE             |X        |1.8          |109     |
|KIA    |Niro FE       |STATION WAGON - SMALL|X        |1.6          |110     |
|HONDA  |ACCORD HYBRID |MID-SIZE             |X        |2.0          |110     |
|KIA    |NIRO FE       |STATION WAGON - 

In [13]:
# checking how co2 emissions change depending on fuel type
# i use avg() & count() => typical values & nº cars per group

from pyspark.sql.functions import avg, count

df_clean.groupBy("fuel_type") \
    .agg(
        count("*").alias("n_cars"),
        avg("co2_g_km").alias("avg_co2")
    ) \
    .orderBy(col("avg_co2").asc()) \
    .show(truncate=False)


+---------+------+------------------+
|fuel_type|n_cars|avg_co2           |
+---------+------+------------------+
|N        |1     |213.0             |
|D        |102   |230.68627450980392|
|X        |1700  |234.63588235294117|
|Z        |1717  |265.221316249272  |
|E        |180   |284.4611111111111 |
+---------+------+------------------+



In [14]:
# checking co2 emissions by vehicle class: like suv, compact, etc...
# this can show trends like bigger cars = higher co2

df_clean.groupBy("vehicle_class") \
    .agg(
        count("*").alias("n_cars"),
        avg("co2_g_km").alias("avg_co2")
    ) \
    .orderBy(col("avg_co2").asc()) \
    .show(20, truncate=False)


+------------------------+------+------------------+
|vehicle_class           |n_cars|avg_co2           |
+------------------------+------+------------------+
|STATION WAGON - SMALL   |140   |203.50714285714287|
|COMPACT                 |515   |218.6             |
|MID-SIZE                |575   |222.10434782608695|
|SUV - SMALL             |579   |236.36614853195164|
|SPECIAL PURPOSE VEHICLE |39    |237.02564102564102|
|MINICOMPACT             |169   |238.75739644970415|
|SUBCOMPACT              |327   |244.29969418960243|
|STATION WAGON - MID-SIZE|28    |250.92857142857142|
|FULL-SIZE               |303   |260.36633663366337|
|MINIVAN                 |35    |260.65714285714284|
|PICKUP TRUCK - SMALL    |64    |278.78125         |
|TWO-SEATER              |252   |282.01984126984127|
|PICKUP TRUCK - STANDARD |259   |301.4131274131274 |
|SUV - STANDARD          |350   |306.6142857142857 |
|VAN - CARGO             |22    |361.5             |
|VAN - PASSENGER         |43    |390.953488372

In [15]:
# looking at engine size vs co2 in a simple way
# i create basic bins to see if bigger engines have higher co2 on average

from pyspark.sql.functions import when

df_bins = df_clean.withColumn(
    "engine_bin",
    when(col("engine_size_l") < 2.0, "<2.0") \
    .when((col("engine_size_l") >= 2.0) & (col("engine_size_l") < 3.0), "2.0-2.9") \
    .when((col("engine_size_l") >= 3.0) & (col("engine_size_l") < 4.0), "3.0-3.9") \
    .otherwise(">=4.0")
)

df_bins.groupBy("engine_bin") \
    .agg(
        count("*").alias("n_cars"),
        avg("co2_g_km").alias("avg_co2")
    ) \
    .orderBy("engine_bin") \
    .show(truncate=False)


+----------+------+------------------+
|engine_bin|n_cars|avg_co2           |
+----------+------+------------------+
|2.0-2.9   |1200  |214.72            |
|3.0-3.9   |1104  |260.71286231884056|
|<2.0      |513   |182.28070175438597|
|>=4.0     |883   |328.6647791619479 |
+----------+------+------------------+



RECOMENDATION SYSTEM ~~~~~~~~~~~~~~

In [16]:
# creating a simple ID per car & readable name
# this helps me show recommendations more clearly

from pyspark.sql.functions import concat_ws, monotonically_increasing_id

df_rec = df_clean.withColumn(
    "car_name",
    concat_ws(" ", col("make"), col("model"))
).withColumn(
    "car_id",
    monotonically_increasing_id()
)

df_rec.select("car_id", "car_name", "vehicle_class", "fuel_type", "engine_size_l", "co2_g_km") \
    .show(5, truncate=False)


+------+------------------------------------------+---------------+---------+-------------+--------+
|car_id|car_name                                  |vehicle_class  |fuel_type|engine_size_l|co2_g_km|
+------+------------------------------------------+---------------+---------+-------------+--------+
|0     |BMW M5                                    |MID-SIZE       |Z        |4.4          |338     |
|1     |BUICK REGAL                               |MID-SIZE       |Z        |2.0          |228     |
|2     |GMC SAVANA 3500 PASSENGER                 |VAN - PASSENGER|E        |6.0          |413     |
|3     |LAND ROVER RANGE ROVER SPORT V8 5.0 SC FFV|SUV - STANDARD |Z        |5.0          |338     |
|4     |AUDI R8                                   |TWO-SEATER     |Z        |4.2          |327     |
+------+------------------------------------------+---------------+---------+-------------+--------+
only showing top 5 rows


In [17]:
# building FEATURES 4 SIMILARITY
# i use onehot for categorical columns & scale numeric columns
# then i create one final vector called "features"

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

cat_cols = ["vehicle_class", "fuel_type", "transmission"]
cat_cols = [c for c in cat_cols if c in df_rec.columns]

num_cols = ["engine_size_l", "cylinders", "fuel_city_l100", "fuel_hwy_l100", "fuel_comb_l100", "fuel_comb_mpg", "co2_g_km"]
num_cols = [c for c in num_cols if c in df_rec.columns]

indexers = [StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid="keep") for c in cat_cols]
encoders = [OneHotEncoder(inputCol=c+"_idx", outputCol=c+"_ohe") for c in cat_cols]

assembler = VectorAssembler(
    inputCols=[c+"_ohe" for c in cat_cols] + num_cols,
    outputCol="raw_features"
)

scaler = StandardScaler(inputCol="raw_features", outputCol="features", withMean=True, withStd=True)

pipe = Pipeline(stages=indexers + encoders + [assembler, scaler])

model_feat = pipe.fit(df_rec)
df_feat = model_feat.transform(df_rec)

df_feat.select("car_id", "car_name", "features").show(3, truncate=False)


+------+-------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
# using lsh to find SIMILARITY CARS based on the FEATURE VECTPR
# this is an approximate nearest neighbor method, so it is fast on bigger datasets

from pyspark.ml.feature import BucketedRandomProjectionLSH

lsh = BucketedRandomProjectionLSH(
    inputCol="features",
    outputCol="hashes",
    bucketLength=2.0,
    numHashTables=3
)

lsh_model = lsh.fit(df_feat)


In [20]:
# selecting 1 CAR to RECOMEND SIMILAR CARS
# i choose by searching a word in the car_name (example: "CIVIC" or "COROLLA")

target_word = "CIVIC"   # the model I choose to test yetsss

target_df = df_feat.filter(col("car_name").contains(target_word)).limit(1)

target_df.select("car_id", "car_name", "vehicle_class", "fuel_type", "co2_g_km").show(truncate=False)


+------+--------------+-------------+---------+--------+
|car_id|car_name      |vehicle_class|fuel_type|co2_g_km|
+------+--------------+-------------+---------+--------+
|51    |HONDA CIVIC Si|COMPACT      |Z        |216     |
+------+--------------+-------------+---------+--------+



In [22]:
# finding NEAREST NEIGHBOURS to the selected car
# the result includes a distance column (minus distance = more similar)

if target_df.count() == 0:
    print("no car found with that word ops")
else:
    recs = lsh_model.approxNearestNeighbors(df_feat, target_df.first()["features"], 10)

    recs.select(
        "car_id", "car_name", "vehicle_class", "fuel_type",
        "engine_size_l", "co2_g_km", "distCol"
    ).show(truncate=False)


+------+--------------------+-------------+---------+-------------+--------+-------------------+
|car_id|car_name            |vehicle_class|fuel_type|engine_size_l|co2_g_km|distCol            |
+------+--------------------+-------------+---------+-------------+--------+-------------------+
|51    |HONDA CIVIC Si      |COMPACT      |Z        |2.4          |216     |0.0                |
|372   |ACURA ILX           |COMPACT      |Z        |2.4          |221     |0.20982139575509712|
|1950  |VOLKSWAGEN BEETLE   |COMPACT      |Z        |2.0          |216     |0.28979835715817404|
|2349  |AUDI A4 QUATTRO     |COMPACT      |Z        |2.0          |214     |0.29666192004333447|
|1801  |VOLKSWAGEN GOLF R   |COMPACT      |Z        |2.0          |220     |0.3016906129624442 |
|3129  |ACURA TSX           |COMPACT      |Z        |2.4          |225     |0.34105995468544054|
|638   |VOLKSWAGEN JETTA GLI|COMPACT      |Z        |2.0          |212     |0.3490533031673379 |
|3298  |BUICK VERANO        |C

In [24]:
# evaluating an idea for example :D
# if cars are really "similar", their co2 values should not be extremely different
# so i be doing min/max co2 inside the recommended list

if target_df.count() != 0:
    recs_small = recs.select("co2_g_km")
    recs_small.describe().show()


+-------+-----------------+
|summary|         co2_g_km|
+-------+-----------------+
|  count|               10|
|   mean|            220.3|
| stddev|5.657836257007717|
|    min|              212|
|    max|              229|
+-------+-----------------+

